In [1]:
!pip install youtube_transcript_api
!pip install pytube
!pip install transformers
!pip install google-api-python-client
!pip install yt-dlp
!pip install pinecone-client
!pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import cv2
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import YouTube
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import os
from pytube import Playlist
from transformers import pipeline
import faiss
import numpy as np
from googleapiclient.discovery import build
from pytube import YouTube, Playlist
from faiss import IndexFlatL2
from yt_dlp import YoutubeDL
from moviepy.editor import VideoFileClip


# Set up YouTube API
developer_api_key = "YOUR_GOOGLE_DEVELOPER_KEY"
youtube = build("youtube", "v3", developerKey=developer_api_key)
openai.api_key = 'YOUR_OPEN_API_KEY'

pinecone_api_key = 'YOUR_PINECONE_API_KEY'

# Initialize CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [ ]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from googleapiclient.discovery import build
import logging

load_dotenv()

# Initialize logging
logging.basicConfig(level=logging.INFO)

from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

def init_pinecone(index_name):
    pc = Pinecone(
        api_key='YOUR_PINECONE_API_KEY'
    )

    # Now do stuff
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=512,
            metric='euclidean',
            spec=spec
            )
    index = pc.Index(index_name)
    return index

In [ ]:

# Function to retrieve video IDs from playlist
def get_playlist_videos(playlist_url):
    playlist = Playlist(playlist_url)
    return [video.video_id for video in playlist.videos]

def extract_frames(video_path, interval=1, max_frames=500):
    cap = cv2.VideoCapture(video_path)
    print(cap)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return []
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print(f"Error: Unable to read frame from video file {video_path}")
            break
        frames.append(frame)
        if len(frames) >= max_frames:
            break
    cap.release()
    if len(frames) == 0:
        print(f"No frames extracted from {video_path}.")
    return frames

def video_to_images(video_path, output_folder, fps=0.2):
    try:
        os.makedirs(output_folder, exist_ok=True)
        clip = VideoFileClip(video_path)
        clip.write_images_sequence(
            os.path.join(output_folder, "frame%04d.png"),
            fps=fps
        )
        print(f"Frames successfully extracted to {output_folder}")
        return [os.path.join(output_folder, f) for f in sorted(os.listdir(output_folder)) if f.endswith(".png")]
    except Exception as e:
        print(f"Error processing video {video_path}: {e}")
        return []

# Retrieve transcript of video
def get_video_transcript(video_id):
    try:
        return YouTubeTranscriptApi.get_transcript(video_id)
    except Exception as e:
        print(f"Error retrieving transcript for video {video_id}: {e}")
        return ""

# Generate multimodal embeddings
def create_multimodal_embeddings(frames, transcript):
    text_inputs = processor(text=transcript, return_tensors="pt", padding=True, truncation=True)
    image_inputs = processor(images=frames, return_tensors="pt", padding=True)

    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
        image_features = model.get_image_features(**image_inputs)

    return text_features, image_features

# Download the video and return its file path
def download_video(url, options):
    with YoutubeDL(options) as ydl:
        info_dict = ydl.extract_info(url, download=True)
        video_path = ydl.prepare_filename(info_dict)  # Get the path of the downloaded video
    return video_path

def get_filename(url, options):
    with YoutubeDL(options) as ydl:
        info_dict = ydl.extract_info(url, download=False)
        video_path = ydl.prepare_filename(info_dict)  # Get the path of the downloaded video
    return video_path

def process_video(video_url, folder='data'):
    print(f"Processing video: {video_url}")

    yt = YouTube(video_url)
    video_id = yt.video_id

    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'outtmpl': os.path.join(folder, '%(title)s.%(ext)s'),
        'quiet': True,
        'merge_output_format': 'mp4',
        'postprocessors': [{'key': 'FFmpegVideoConvertor', 'preferedformat': 'mp4'}]
    }
    video_title = yt.title
    video_path = get_filename(video_url, ydl_opts)
    if not os.path.exists(video_path):
        video_path = download_video(video_url, ydl_opts)
    print(f"Downloaded video path: {video_path}")

    # Extract frames
    frame_folder = os.path.join(folder, video_id)
    if not os.path.exists(frame_folder):
        os.makedirs(frame_folder)
        frames = video_to_images(video_path, frame_folder)
    else:
        frames = [os.path.join(frame_folder, f) for f in sorted(os.listdir(frame_folder)) if f.endswith(".png")]

    if not frames:
        print(f"No frames extracted for video {video_url}. Skipping.")
        return None, None, None, None

    # Get transcript
    transcript = get_video_transcript(video_id)
    if not transcript:
        print(f"No transcript available for video {video_url}. Skipping.")
        return None, None, video_title

    return transcript, frames, video_title

def chunk_transcript(transcript, max_length=1000):
    """
    Chunk transcript into smaller pieces for better indexing.

    Parameters:
    transcript (str): Full transcript text.
    max_length (int): Maximum length of each chunk.

    Returns:
    list of dict: List of chunks with their corresponding timestamps.
    """
    chunks = []
    current_chunk = []
    current_length = 0
    start_time = None

    for entry in transcript:
        if current_length == 0:
            start_time = entry["start"]  # Use the start time of the first word in the chunk
        current_chunk.append(entry["text"])
        current_length += len(entry["text"].split())  # Count words in the text

        if current_length >= max_length:
            chunks.append({
                "text": " ".join(current_chunk),
                "timestamp": f"{start_time:.2f}s"  # Format start time as seconds
            })
            current_chunk = []
            current_length = 0

    # Add the last chunk if it exists
    if current_chunk:
        chunks.append({
            "text": " ".join(current_chunk),
            "timestamp": f"{start_time:.2f}s"
        })
    return chunks

def store_embeddings_in_pinecone(frame_paths, transcript, video_id, index='video-rag', batch_size=16):
    """
    Store embeddings in a single Pinecone namespace with metadata for filtering.

    Parameters:
    frame_paths (list of str): Paths to extracted frames.
    transcript (str): Transcript text.
    video_id (str): Unique identifier for the video.
    batch_size (int): Number of frames to process per batch.
    """
    try:
        # Load all images
        images = [Image.open(frame_path) for frame_path in frame_paths]

        # Generate and store text embedding
        chunks = chunk_transcript(transcript, max_length=200)  # Split transcript into chunks
        for i, chunk in enumerate(chunks):
            text_inputs = processor(text=chunk["text"], return_tensors="pt", padding=True, truncation=True)
            with torch.no_grad():
                text_features = model.get_text_features(**text_inputs).cpu().numpy()

            metadata = {
                "type": "text",
                "video_id": video_id,
                "timestamp": chunk["timestamp"],
                "content": chunk["text"]
            }
            index.upsert([(f"text-{video_id}-{i}", text_features[0], metadata)])

        # Generate and store image embeddings in batches
        for i in range(0, len(images), batch_size):
            image_batch = images[i:i + batch_size]
            image_inputs = processor(images=image_batch, return_tensors="pt", padding=True)
            with torch.no_grad():
                image_features = model.get_image_features(**image_inputs).cpu().numpy()

            # Create metadata for each frame
            vectors = []
            for j, feature in enumerate(image_features):
                frame_metadata = {
                    "type": "image",
                    "video_id": video_id,
                    "frame_index": i + j,
                    "timestamp": f"{(i + j) * 5}s"  # Assuming 1 frame per 5 seconds
                }
                vectors.append((f"image-{video_id}-{i + j}", feature, frame_metadata))

            index.upsert(vectors)

        print(f"Stored embeddings for video: {video_id}")
    except Exception as e:
        print(f"Error storing embeddings for video {video_id}: {e}")


# Process all videos in a playlist
def process_playlist(playlist_url):
    index = init_pinecone('video-rag')
    video_ids = get_playlist_videos(playlist_url)
    data_folder = '/content/drive/MyDrive/Video_RAG/Data'
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    for video_id in video_ids:
        try:
            print("Processing:"+str(video_id))
            trans, frms = process_video(f"https://www.youtube.com/watch?v={video_id}", data_folder)
            store_embeddings_in_pinecone(frms, trans, video_id, index)
            print("Embeddings created for "+str(video_id))
        except Exception as e:
            print(f"Error processing video {video_id}: {e}")
    return video_ids

playlist_url = "https://www.youtube.com/playlist?list=PL6299F3195349CCDA"
# Process playlist
video_ids = process_playlist(playlist_url)

In [ ]:
import openai
import pinecone
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# CLIP initialization
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def query_pinecone_with_timestamps(question, top_k=5):
    """
    Query Pinecone to fetch relevant embeddings and metadata based on the question.
    """
    # Generate query embedding
    question_inputs = processor(text=question, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        query_embedding = model.get_text_features(**question_inputs).cpu().numpy()

    # Query Pinecone for relevant embeddings
    results = index.query(vector=query_embedding[0].tolist(), top_k=top_k, include_metadata=True, include_values=True)
    # Extract relevant context
    context_segments = []
    for match in results["matches"]:
        metadata = match["metadata"]
        if metadata["type"] == "text":
            context_segments.append({
                "type": "text",
                "content": metadata.get("content"),
                "timestamp": metadata.get("timestamp"),
                "video_id": metadata.get("video_id")
            })
        elif metadata["type"] == "image":
            context_segments.append({
                "type": "image",
                "frame_index": metadata.get("frame_index"),
                "video_id": metadata.get("video_id"),
                "timestamp": metadata.get("timestamp")
            })
    return context_segments

def generate_gpt4_answer(question, context_segments, frame_folder):
    """
    Generate an answer using OpenAI GPT-4 based on the retrieved context.
    """
    # Build the prompt with text and image descriptions
    prompt = f"Question: {question}\n\nRelevant Context:\n"
    for segment in context_segments:
        if segment["type"] == "text":
            prompt += f"[Text at {segment['timestamp']}]: {segment['content']}\n"
        elif segment["type"] == "image":
            # Generate a description of the image (simplified here for illustration)
            frame_folder = os.path.join(frame_folder, segment['video_id'])
            image_path = f"{frame_folder}/frame{segment['frame_index']:04d}.png"
            prompt += f"[Image at {segment['timestamp']}]: See the frame extracted from the video.\n"

    prompt += "\nAnswer:"

    # Query GPT-4 to generate an answer
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions based on video frames and transcripts."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

In [ ]:
# Example Workflow
question = "What are life history traits?"

# Query Pinecone
context_segments = query_pinecone_with_timestamps(question)

# Generate Answer
frame_folder = 'Data'
answer = generate_gpt4_answer(question, context_segments, frame_folder)

print("Generated Answer:")
print(answer)